### Tensors and Operations

In [23]:
import tensorflow as tf
import numpy as np
import tensorflow.keras as keras

In [2]:
t = tf.constant([[1.,2.,3.],[4.,5.,6.]])  #matrix

In [3]:
tf.constant(45) # scalar

<tf.Tensor: shape=(), dtype=int32, numpy=45>

In [4]:
t.shape

TensorShape([2, 3])

In [5]:
t.dtype

tf.float32

In [6]:
t[:,1:]

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[2., 3.],
       [5., 6.]], dtype=float32)>

In [7]:
t[...,1,tf.newaxis]

<tf.Tensor: shape=(2, 1), dtype=float32, numpy=
array([[2.],
       [5.]], dtype=float32)>

In [8]:
a = np.array([2.,4.,5.])
tf.constant(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([2., 4., 5.])>

In [9]:
t.numpy()

array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)

In [10]:
tf.square(a)

<tf.Tensor: shape=(3,), dtype=float64, numpy=array([ 4., 16., 25.])>

In [11]:
np.square(t)

array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)

### Type Conversions

In [12]:
tf.constant(2.) + tf.constant(40,dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [13]:
t2 = tf.constant(40,dtype=tf.float64)

In [14]:
tf.constant(2.) + tf.cast(t2,dtype=tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

### Variables

In [15]:
v = tf.Variable([[1.,2.,3.],[4.,5.,6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [16]:
v.assign(2 * v)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [17]:
v[0,1].assign(42)

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [18]:
v[:,2].assign([0.,1.])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [19]:
v.scatter_nd_update(indices=[[0,0],[1,2]],updates=[100,200])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

### Customizing Models and Training Algorithms

#### Custom Loss functions

In [20]:
def huber_fn(y_true,y_pred):
    error =  y_true - y_pred
    is_small_error = tf.abs(error) < 1
    squared_loss = tf.square(error) / 2
    linear_loss = tf.abs(error) - 0.5
    
    return tf.where(is_small_error,squared_loss,linear_loss)

### Saving and Loading Models That Contain Custom Components

In [24]:
model = keras.models.load_model("my_keras_model.h5",custom_objects={"huber_fn":huber_fn})

In [25]:
#### Different threshold
def create_huber(threshold=1.0):
    def huber_fn(y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error)/2
        linear_loss = threshold * tf.abs(error) - threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    return huber_fn

In [26]:
model.compile(loss = create_huber(1.0),optimizer="nadam")

In [27]:
model =  keras.models.load_model

In [29]:
class HuberLoss(keras.losses.Loss):
    def __init__(self,threshold=1.0, **kwargs):
        self.threshold = threshold
        super().__init__(**kwargs)
    def call(self,y_true,y_pred):
        error = y_true - y_pred
        is_small_error = tf.abs(error) < threshold
        squared_loss = tf.square(error)/2
        linear_loss = threshold * tf.abs(error) - threshold**2/2
        return tf.where(is_small_error,squared_loss,linear_loss)
    def get_config(self):
        base_config = super().get_config()
        return {**base_config,"threshold":self.threshold}
        

### Custom Activation Functions, Initializers, Regularizers, and Constraints

In [31]:
def my_softplus(z): # return value is just tf.nn.softplus(z)
    return tf.math.log(tf.exp(z)+1.0)

def my_glorot_initializer(shape,dtype=tf.float32):
    stddev = tf.sqrt(2./ (shape[0]+shape[1]))
    return tf.random.normal(shape,stddev=stddev,dtype=dtype)

def my_l1_regularizer(weights):
    return tf.reduce_sum(tf.abs(0.01 * weights))

def my_positive_weights(weights): # return value is just tf.nn.relu(weights)
    return tf.where(weights < 0. ,tf.zeros_like(weights),weights)

In [32]:
layer = keras.layers.Dense(30,activation=my_softplus,kernel_initializer=my_glorot_initializer,
                          kernel_constraint=my_positive_weights,
                          kernel_regularizer=my_l1_regularizer)

In [33]:
### Saving Hyperparmeters while saving the model
class MyL1Regularizer(keras.regularizers.Regularizer):
    def __init__(self,factor):
        self.factor = factor
    def __call__(self,weights):
        return tf.reduce_sum(tf.abs(factor * weights))
    def get_config(self):
        return {"factor":self.factor}

### Custom Metrics